In [ ]:
'''
README

This is my sourse code and results of assignment 1 and it is divided into X parts:
    a.Data preparation
      *This part is mainly about how to import the data from text file and integrate them into a DataFrame object.
      *I define a function to handle this work.
      *I import all of the data from text files and show it in the 6th block for you check.
      *Please check the notes in every blocks for details.
    b.Models
      *I make 8 models according to the assignment requirement and put each of them in seperate blocks.
      *Each blocks of different models is made of the training part and the evaluation part.
      *Please check the notes in every blocks for details.
'''

In [1]:
import os
import codecs
import glob
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, precision_score, recall_score
import pickle

In [2]:
'''
    As for this function:
    a.Import text files from a folder
    b.Give a value to the "type" column which "1" means positive comment and "0" means negtive comment
    c.Create a DataFrame object for the next step
'''
def addData(s):
    os.chdir(s)
    files = glob.glob('*.txt')        
    b=[]
    for i in files:
        f = open(i,'rb')
        a=f.read()
        b.append(a)
    comdf=pd.DataFrame(b)
    path = os.getcwd()
    if path[-3:] == 'neg':
        comdf['type']=0
    elif path[-3:] == 'pos':
        comdf['type']=1
    comdf.rename(columns={0:'comment'},inplace=True)
    return comdf

In [3]:
'''
Import all of the text data to a Dataframe object
'''
df_all=pd.DataFrame(columns=['comment','type'])
df_all[['type']] = df_all[['type']].astype(int)
n1=addData('E:\\document\\strange\\aclImdb\\test\\neg')
n2=addData('E:\\document\\strange\\aclImdb\\test\\pos')
n3=addData('E:\\document\\strange\\aclImdb\\train\\neg')
n4=addData('E:\\document\\strange\\aclImdb\\train\\pos')
df_all=df_all.append(n1)
df_all=df_all.append(n2)
df_all=df_all.append(n3)
df_all=df_all.append(n4)

In [4]:
'''
Show the DataFrame
'''
df_all

,comment,type
0,"b""Once again Mr. Costner has dragged out a mov...",0
1,"b""This is an example of why the majority of ac...",0
2,"b""First of all I hate those moronic rappers, w...",0
3,"b""Not even the Beatles could write songs every...",0
4,"b""Brass pictures (movies is not a fitting word...",0
5,b'A funny thing happened to me while watching ...,0
6,"b""This German horror film has to be one of the...",0
7,"b'Being a long-time fan of Japanese film, I ex...",0
8,"b'""Tokyo Eyes"" tells of a 17 year old Japanese...",0
9,"b""Wealthy horse ranchers in Buenos Aires have ...",0


In [5]:
'''
splitting in train and test stes 
'''
X = df_all["comment"]
y = df_all["type"]
X_train,X_test,y_train, y_test = train_test_split(
        X,y,test_size=0.3, stratify=y, random_state=100)

In [6]:
'''
CounterVectorizer + Naive Bayes
'''

'''Model training'''
clf = Pipeline([
        ('vec',CountVectorizer()),
        ('nb',MultinomialNB())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.8447
Precision: 0.8702
Recall   : 0.8101


In [8]:
with open('CVNB.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [9]:
'''
TfidfVectorizer + Naive Bayes
'''

'''Model training'''
clf = Pipeline([
        ('vec',TfidfVectorizer()),
        ('nb',MultinomialNB())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.8579
Precision: 0.8804
Recall   : 0.8284


In [10]:
with open('TVNB.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [11]:
'''
CounterVectorizer + Logistic Regression
'''

'''Model training'''
clf = Pipeline([
        ('vec',CountVectorizer()),
        ('lr',LogisticRegression())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.8873
Precision: 0.8848
Recall   : 0.8907


In [12]:
with open('CVLR.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [13]:
'''
TfidfVectorizer + Logistic Regression
'''

'''Model training'''
clf = Pipeline([
        ('vec',TfidfVectorizer()),
        ('lr',LogisticRegression())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.8953
Precision: 0.8880
Recall   : 0.9047


In [14]:
with open('TVLR.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [15]:
'''
CounterVectorizer + Naive Bayes + bigram
'''

'''Model training'''
clf = Pipeline([
        ('vec',CountVectorizer(ngram_range=(1,2))),
        ('nb',MultinomialNB())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.8799
Precision: 0.8949
Recall   : 0.8609


In [16]:
with open('CVNBbigram.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [17]:
'''
TfidfVectorizer + Naive Bayes + bigram
'''

'''Model training'''
clf = Pipeline([
        ('vec',TfidfVectorizer(ngram_range=(1,2))),
        ('nb',MultinomialNB())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.8889
Precision: 0.9098
Recall   : 0.8635


In [18]:
with open('TVNBbigram.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [19]:
'''
CounterVectorizer + Logistic Regression + bigram
'''

'''Model training'''
clf = Pipeline([
        ('vec',CountVectorizer(ngram_range=(1,2))),
        ('lr',LogisticRegression())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.9071
Precision: 0.8997
Recall   : 0.9163


In [20]:
with open('CVLRbigram.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [21]:
'''
TfidfVectorizer + Logistic Regression + bigram
'''

'''Model training'''
clf = Pipeline([
        ('vec',TfidfVectorizer(ngram_range=(1,2))),
        ('lr',LogisticRegression())
        ])
clf.fit(X_train, y_train)

'''Evaluation'''
y_pred = clf.predict(X_test)
print("AUC      : {:.4f}".format(roc_auc_score(y_test,y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test,y_pred)))
print("Recall   : {:.4f}".format(recall_score(y_test,y_pred)))

AUC      : 0.8960
Precision: 0.8886
Recall   : 0.9055


In [22]:
with open('TVLRbigram.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

In [ ]:
'''
This is the form of 9 different models' performance below:
 _________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
|          |CounterVectorizer + Naive Bayes  |  TfidfVectorizer + Naive Bayes  |  CounterVectorizer + Logistic Regression  |  TfidfVectorizer + Logistic Regression  |  CounterVectorizer + Naive Bayes + bigram  |  TfidfVectorizer + Naive Bayes + bigram  |  CounterVectorizer + Logistic Regression + bigram  |  TfidfVectorizer + Logistic Regression + bigram  |  fastText  |
 ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| AUC      |0.8447                           |0.8579                           |0.8873                                     |0.8953                                   |0.8799                                      |0.8889                                    |0.9071                                              |0.8960                                            |None        |
 --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
| Precision|0.8702                           |0.8804                           |0.8848                                     |0.8880                                   |0.8949                                      |0.9098                                    |0.8997                                              |0.8886                                            |0.8952      |
 ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| Recall   |0.8101                           |0.8284                           |0.8907                                     |0.9047                                   |0.8609                                      |0.8635                                    |0.9163                                              |0.9055                                            |0.8952      |
 ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"CounterVectorizer + Logistic Regression + bigram" model perform best on AUC
"TfidfVectorizer + Naive Bayes + bigram" model perform best on Precision
"CounterVectorizer + Logistic Regression + bigram" model perform best on Recall
'''